In [4]:
import pandas as pd
import sqlite3
import os

# import raw data file path
FILE_PATH = "../data/Churn_Modelling.csv"
DB_NAME = "../data/churn.db"

def create_churn_database():
    # 1. Read raw data
    if not os.path.exists(FILE_PATH):
        print(f"Error: Can't find file {FILE_PATH}")
        return

    df = pd.read_csv(FILE_PATH)
    print(f"Load raw data successfully， {len(df)} rows in total.")

    # 2. Connect to SQLite database
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Table 1: lookup_geography 
    cursor.execute("DROP TABLE IF EXISTS lookup_geography")
    cursor.execute("""
        CREATE TABLE lookup_geography (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            geography_name TEXT UNIQUE
        )
    """)

    # Get unique geographies
    unique_geos = df['Geography'].unique()
    geo_data = [(str(x),) for x in unique_geos]
    cursor.executemany("INSERT OR IGNORE INTO lookup_geography (geography_name) VALUES (?)", geo_data)
    
    # Build a mapping from geography name to id
    geo_df = pd.read_sql("SELECT * FROM lookup_geography", conn)
    geo_map = dict(zip(geo_df['geography_name'], geo_df['id']))
    df['geo_id'] = df['Geography'].map(geo_map) # Add geo_id column to main df
    print("✅ Table lookup_geography created successfully")

    # Table 2: customer_info
    cursor.execute("DROP TABLE IF EXISTS customer_info")
    cursor.execute("""
        CREATE TABLE customer_info (
            customer_id INTEGER PRIMARY KEY,
            surname TEXT,
            gender TEXT,
            age INTEGER,
            geo_id INTEGER,
            FOREIGN KEY (geo_id) REFERENCES lookup_geography (id)
        )
    """)

    # Select relevant columns for customer_info
    cust_cols = ['CustomerId', 'Surname', 'Gender', 'Age', 'geo_id']
    customer_data = df[cust_cols].copy()
    customer_data.columns = ['customer_id', 'surname', 'gender', 'age', 'geo_id']
    customer_data.to_sql('customer_info', conn, if_exists='append', index=False)
    print("✅ Table customer_info created successfully")

    # Table 3: credit_stats
    cursor.execute("DROP TABLE IF EXISTS credit_stats")
    cursor.execute("""
        CREATE TABLE credit_stats (
            customer_id INTEGER,
            credit_score INTEGER,
            tenure INTEGER,
            balance REAL,
            num_of_products INTEGER,
            has_cr_card INTEGER,
            is_active_member INTEGER,
            estimated_salary REAL,
            exited INTEGER,
            FOREIGN KEY (customer_id) REFERENCES customer_info (customer_id)
        )
    """)
    # Select relevant columns for credit_stats
    stats_cols = ['CustomerId', 'CreditScore', 'Tenure', 'Balance', 'NumOfProducts', 
                  'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']
    stats_data = df[stats_cols].copy()
    stats_data.columns = ['customer_id', 'credit_score', 'tenure', 'balance', 'num_of_products', 
                          'has_cr_card', 'is_active_member', 'estimated_salary', 'exited']
    stats_data.to_sql('credit_stats', conn, if_exists='append', index=False)
    print("✅ Table credit_stats created successfully")


    # Join Back
    query = """
    SELECT 
        c.customer_id, c.surname, g.geography_name, 
        s.credit_score, s.balance, s.exited
    FROM customer_info c
    JOIN credit_stats s ON c.customer_id = s.customer_id
    JOIN lookup_geography g ON c.geo_id = g.id
    """
    df_restored = pd.read_sql(query, conn)
    print("-" * 30)
    print("Passing Test")
    print(df_restored.head(3))
    
    conn.commit()
    conn.close()

# execute
create_churn_database()

Load raw data successfully， 10000 rows in total.
✅ Table lookup_geography created successfully
✅ Table customer_info created successfully
✅ Table credit_stats created successfully
------------------------------
Passing Test
   customer_id   surname geography_name  credit_score    balance  exited
0     15634602  Hargrave         France           619       0.00       1
1     15647311      Hill          Spain           608   83807.86       0
2     15619304      Onio         France           502  159660.80       1
